<a href="https://colab.research.google.com/github/andrewolf80550/AA/blob/main/PrediccionReingresoStreamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [11]:
!ngrok config add-authtoken 2loQTFlZIxTTAKqyt51suDo4sof_73D38PUAzeerKTMraaAGC

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [16]:
# Cachear los datos
@st.cache_data
def cargar_datos():
    # Simulación de datos para ejemplo
    df_large = pd.DataFrame({
        'Genero': np.random.choice(['Masculino', 'Femenino'], size=100),
        'Actividad_Fisica': np.random.choice(['Alta', 'Moderada', 'Baja'], size=100),
        'Salud_Mental': np.random.choice(['Buena', 'Moderada', 'Mala'], size=100),
        'Edad': np.random.randint(18, 100, size=100),
        'Duracion_Hospitalizacion_Dias': np.random.randint(1, 40, size=100),
        'Numero_Visitas_Urgencias_Ultimo_Año': np.random.randint(0, 10, size=100),
        'IMC': np.random.uniform(15, 40, size=100),
        'Reingreso_Predicho': np.random.choice([0, 1], size=100)
    })
    return df_large

# Cachear el modelo
@st.cache_resource
def entrenar_modelo(X_train_preprocessed, y_train):
    xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

    # Definir los hiperparámetros para optimización
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 6, 10],
        'learning_rate': [0.01, 0.1, 0.3],
        'subsample': [0.8, 1.0]
    }

    # Configurar el modelo para búsqueda de los mejores hiperparámetros
    grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

    # Entrenar el modelo con validación cruzada
    grid_search.fit(X_train_preprocessed, y_train)

    return grid_search.best_estimator_


2024-09-10 01:14:40.402 No runtime found, using MemoryCacheStorageManager


In [17]:
if st.button('Entrenar Modelo'):
    modelo = entrenar_modelo(X_train_preprocessed, y_train)
    st.write("Modelo entrenado con éxito.")

# Predecir sin volver a entrenar el modelo
if st.button("Predecir"):
    probabilidad_reingreso = modelo.predict_proba(nuevo_paciente_preprocesado)[0][1]
    st.write(f"La probabilidad de reingreso del paciente es: {probabilidad_reingreso:.2f}")


2024-09-10 01:14:59.041 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 01:14:59.045 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 01:14:59.048 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 01:14:59.050 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 01:14:59.054 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 01:14:59.057 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 01:14:59.059 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 01:14:59.062 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [18]:
df_large = df_large.astype({
    'Edad': 'int32',
    'Duracion_Hospitalizacion_Dias': 'int32',
    'Numero_Visitas_Urgencias_Ultimo_Año': 'int32',
    'IMC': 'float32'
})


In [19]:
if st.button('Mostrar Importancia de Características'):
    fig, ax = plt.subplots()
    ax.barh(features, importances)
    ax.set_xlabel('Importancia')
    ax.set_title('Importancia de las Características en el Modelo XGBoost')
    st.pyplot(fig)


2024-09-10 01:15:28.550 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 01:15:28.553 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 01:15:28.555 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 01:15:28.558 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [20]:
param_grid = {
    'n_estimators': [10, 50],  # Menos estimadores durante las pruebas
    'max_depth': [3, 6],       # Menor profundidad de árbol para pruebas rápidas
    'learning_rate': [0.1],
    'subsample': [0.8]
}


In [21]:
st.sidebar.button('Actualizar manualmente')


2024-09-10 01:15:58.001 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 01:15:58.003 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 01:15:58.006 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 01:15:58.007 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


False

In [22]:
# Instalar las dependencias necesarias
!pip install streamlit pyngrok xgboost scikit-learn matplotlib

# Crear un archivo app.py que contiene tu aplicación Streamlit
with open('app.py', 'w') as f:
    f.write("""
import streamlit as st
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

# Título de la aplicación
st.title("Predicción de Reingreso Hospitalario con XGBoost")

# Simulación de datos para ejemplo
st.subheader("Datos Simulados")
df_large = pd.DataFrame({
    'Genero': np.random.choice(['Masculino', 'Femenino'], size=100),
    'Actividad_Fisica': np.random.choice(['Alta', 'Moderada', 'Baja'], size=100),
    'Salud_Mental': np.random.choice(['Buena', 'Moderada', 'Mala'], size=100),
    'Edad': np.random.randint(18, 100, size=100),
    'Duracion_Hospitalizacion_Dias': np.random.randint(1, 40, size=100),
    'Numero_Visitas_Urgencias_Ultimo_Año': np.random.randint(0, 10, size=100),
    'IMC': np.random.uniform(15, 40, size=100),
    'Reingreso_Predicho': np.random.choice([0, 1], size=100)
})

# Mostrar una tabla de los primeros 5 registros de los datos simulados
st.write(df_large.head())

# Incluir la variable de género en las columnas categóricas
categorical_cols = ['Genero', 'Actividad_Fisica', 'Salud_Mental']
numerical_cols = ['Edad', 'Duracion_Hospitalizacion_Dias', 'Numero_Visitas_Urgencias_Ultimo_Año', 'IMC']

# Separar variables dependientes e independientes
X = df_large[categorical_cols + numerical_cols]
y = df_large['Reingreso_Predicho']

# Preprocesamiento de los datos: estandarizar numéricos y aplicar OneHotEncoding a categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first'), categorical_cols)
    ])

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocesar los datos
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# --------------------- XGBoost ---------------------
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Definir los hiperparámetros para optimización
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 1.0]
}

# Configurar el modelo para búsqueda de los mejores hiperparámetros
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Entrenar el modelo con validación cruzada
grid_search.fit(X_train_preprocessed, y_train)

# Obtener los mejores parámetros
best_params = grid_search.best_params_
st.write(f"Mejores parámetros encontrados: {best_params}")

# Evaluación final con los mejores parámetros
best_xgb_model = grid_search.best_estimator_
y_pred_xgb_best = best_xgb_model.predict(X_test_preprocessed)

# Calcular la precisión y otras métricas
st.subheader("Evaluación del Modelo Optimizado")
st.text(f"Precisión optimizada del modelo: {accuracy_score(y_test, y_pred_xgb_best):.2f}")
st.text(classification_report(y_test, y_pred_xgb_best))

# Graficar la importancia de las características
st.subheader("Importancia de las Características")
importances = best_xgb_model.feature_importances_
features = numerical_cols + list(preprocessor.named_transformers_['cat'].get_feature_names_out())

fig, ax = plt.subplots()
ax.barh(features, importances)
ax.set_xlabel('Importancia')
ax.set_title('Importancia de las Características en el Modelo XGBoost')
st.pyplot(fig)

# --------------------- Predicción con nuevos datos ---------------------
st.subheader("Predicción para un Nuevo Paciente")

# Función para obtener los datos de entrada del usuario
def obtener_datos_paciente():
    edad = st.number_input("Edad (entre 18 y 100):", min_value=18, max_value=100, value=30)
    duracion_hospitalizacion = st.number_input("Duración de la Hospitalización (días, entre 1 y 40):", min_value=1, max_value=40, value=5)
    num_visitas_urgencias = st.number_input("Número de Visitas a Urgencias (último año, entre 0 y 10):", min_value=0, max_value=10, value=1)
    genero = st.selectbox("Género", ['Masculino', 'Femenino'])
    actividad_fisica = st.selectbox("Actividad Física", ['Alta', 'Moderada', 'Baja'])
    imc = st.number_input("Índice de Masa Corporal (IMC, entre 15 y 40):", min_value=15.0, max_value=40.0, value=25.0)
    salud_mental = st.selectbox("Salud Mental", ['Buena', 'Moderada', 'Mala'])

    # Crear un DataFrame con los datos ingresados
    nuevo_paciente = pd.DataFrame({
        'Edad': [edad],
        'Duracion_Hospitalizacion_Dias': [duracion_hospitalizacion],
        'Numero_Visitas_Urgencias_Ultimo_Año': [num_visitas_urgencias],
        'Genero': [genero],
        'Actividad_Fisica': [actividad_fisica],
        'IMC': [imc],
        'Salud_Mental': [salud_mental]
    })

    return nuevo_paciente

# Obtener datos del nuevo paciente
nuevo_paciente = obtener_datos_paciente()

if st.button("Predecir"):
    # Preprocesar los datos del nuevo paciente
    nuevo_paciente_preprocesado = preprocessor.transform(nuevo_paciente)

    # Predecir la probabilidad de reingreso con el modelo XGBoost
    probabilidad_reingreso = best_xgb_model.predict_proba(nuevo_paciente_preprocesado)[0][1]  # Probabilidad de reingreso (clase 1)

    st.write(f"La probabilidad de reingreso del paciente es: {probabilidad_reingreso:.2f}")
    """)

# Iniciar el túnel ngrok en HTTP (asegúrate de usar tu propio authtoken de ngrok)
from pyngrok import ngrok

# Authtoken para ngrok (reemplaza YOUR_NGROK_AUTH_TOKEN con tu token)
!ngrok config add-authtoken 2loQTFlZIxTTAKqyt51suDo4sof_73D38PUAzeerKTMraaAGC

# Iniciar el túnel ngrok para el puerto de Streamlit
public_url = ngrok.connect(8501)
print(f"URL pública de la aplicación: {public_url}")

# Ejecutar Streamlit en segundo plano
!streamlit run app.py &


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [01:35:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [01:35:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [01:35:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [01:35:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [01:35:17] WARNING: /worksp